In [1]:
%load_ext autoreload

In [2]:
%autoreload

import os
import json
import pandas as pd
import ast
#import sys
#sys.path.append('../')
import nltk
import stanza
from ast import literal_eval
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/mnt/projects/OpenNRE/benchmark/raw_semeval"
OUTPUT_PATH = "/mnt/projects/OpenNRE/benchmark/semeval2010"
outdir = 'original/'
indir = 'original/'
outdir1 = 'entity_blinding/'
outdir2 = 'punct_stop_digit/'
outdir3 = 'punct_digit/'
outdir4 = 'ner_blinding/'
def res(path): return os.path.join(RESOURCE_PATH, path)
def out(path): return os.path.join(OUTPUT_PATH, path)
from opennre.dataset.converters.converter_semeval2010 import get_dataset_dataframe, write_dataframe,\
read_dataframe, check_equality_of_written_and_read_df, write_into_txt, write_relations_json
from opennre.dataset.preprocess import replace_with_concept, replace_digit_punctuation_stop_word,\
get_entity_positions_and_replacement_sentence, preprocess, replace_ner

def makedir(outdir, out):
    if not os.path.exists(out(outdir)):
        os.makedirs(out(outdir))

2021-06-17 19:51:22 INFO: Downloading default packages for language: en (English)...
2021-06-17 19:51:22,607 - stanza - INFO - Downloading default packages for language: en (English)...
2021-06-17 19:51:23 INFO: File exists: /home/igor/stanza_resources/en/default.zip.
2021-06-17 19:51:23,302 - stanza - INFO - File exists: /home/igor/stanza_resources/en/default.zip.
2021-06-17 19:51:27 INFO: Finished downloading models and saved to /home/igor/stanza_resources.
2021-06-17 19:51:27,238 - stanza - INFO - Finished downloading models and saved to /home/igor/stanza_resources.
[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nlp = stanza.Pipeline(lang='en', processors="tokenize")

2021-06-17 19:51:27 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-17 19:51:27,624 - stanza - INFO - Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-17 19:51:27 INFO: Use device: gpu
2021-06-17 19:51:27,661 - stanza - INFO - Use device: gpu
2021-06-17 19:51:27 INFO: Loading: tokenize
2021-06-17 19:51:27,662 - stanza - INFO - Loading: tokenize
2021-06-17 19:51:29 INFO: Done loading processors!
2021-06-17 19:51:29,768 - stanza - INFO - Done loading processors!


In [4]:
df_train = get_dataset_dataframe(nlp, res('TRAIN_FILE.TXT'))

100%|██████████| 8000/8000 [00:55<00:00, 144.75it/s]


In [5]:
df_test = get_dataset_dataframe(nlp, res('TEST_FILE_FULL.TXT'))

100%|██████████| 2717/2717 [00:18<00:00, 143.27it/s]


In [6]:
df_test

,original_sentence,e1,e2,relation_type,metadata,tokenized_sentence
0,The most common audits were about waste and re...,audits,waste,"Message-Topic(e1,e2)","{'e1': {'word': 'audits', 'word_index': [(3, 3...",The most common audits were about waste and re...
1,The company fabricates plastic chairs .,company,chairs,"Product-Producer(e2,e1)","{'e1': {'word': 'company', 'word_index': [(1, ...",The company fabricates plastic chairs .
2,The school master teaches the lesson with a st...,master,stick,"Instrument-Agency(e2,e1)","{'e1': {'word': 'master', 'word_index': [(2, 2...",The school master teaches the lesson with a st...
3,The suspect dumped the dead body into a local ...,body,reservoir,"Entity-Destination(e1,e2)","{'e1': {'word': 'body', 'word_index': [(5, 5)]...",The suspect dumped the dead body into a local ...
4,Avian influenza is an infectious disease of bi...,influenza,virus,"Cause-Effect(e2,e1)","{'e1': {'word': 'influenza', 'word_index': [(1...",Avian influenza is an infectious disease of bi...
...,...,...,...,...,...,...
2712,"After seating all the idols, which itself take...",king,broom,"Instrument-Agency(e2,e1)","{'e1': {'word': 'king', 'word_index': [(13, 13...","After seating all the idols , which itself tak..."
2713,The minister attributed the slow production of...,materials,industries,"Product-Producer(e1,e2)","{'e1': {'word': 'materials', 'word_index': [(8...",The minister attributed the slow production of...
2714,The umbrella frame is provided with a movable ...,umbrella,frame,"Component-Whole(e2,e1)","{'e1': {'word': 'umbrella', 'word_index': [(1,...",The umbrella frame is provided with a movable ...
2715,Manos: The Hands of Fate is a low-budget horro...,film,salesman,"Product-Producer(e1,e2)","{'e1': {'word': 'film', 'word_index': [(12, 12...",Manos : The Hands of Fate is a low - budget ho...


In [7]:
len(df_train)

8000

In [8]:
len(df_test)

2717

In [9]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        metadata = row.metadata
        e1 = metadata['e1']['word_index']
        e2 = metadata['e2']['word_index']
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        original_sentence = df.iloc[index].original_sentence
        tokenized_sentence = df.iloc[index].tokenized_sentence
        metadata = df.iloc[index].metadata
        empty_rows_array.append([index, original_sentence, e1, e2, metadata, tokenized_sentence])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'original_sentence' , 'e1', 'e2', 'metadata', 'tokenized_sentence'])
    return empty_rows_array, new_df

In [10]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [11]:
get_empty_vals(df_train)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [12]:
get_empty_vals(df_test)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [13]:
if not os.path.exists(out(outdir)):
    os.makedirs(out(outdir))

In [14]:
write_dataframe(df_train, out(outdir + 'semeval2010_train_original.csv'))

In [15]:
df_train_copy = read_dataframe(out(outdir + 'semeval2010_train_original.csv'))

In [16]:
# The first checks with the pd.equals method, and the other does a manual checking per column
check_equality_of_written_and_read_df(df_train, df_train_copy)

(True, True)

In [17]:
write_dataframe(df_test, out(outdir + 'semeval2010_val_original.csv'))

In [18]:
df_test_copy = read_dataframe(out(outdir + 'semeval2010_val_original.csv'))

In [19]:
check_equality_of_written_and_read_df(df_test, df_test_copy)

(True, True)

In [20]:
write_into_txt(df_train, out(outdir + 'semeval2010_train_original.txt'))

100%|██████████| 8000/8000 [00:00<00:00, 8613.69it/s]


In [21]:
write_into_txt(df_test, out(outdir + 'semeval2010_val_original.txt'))

100%|██████████| 2717/2717 [00:00<00:00, 8332.75it/s]


In [22]:
nlp = stanza.Pipeline(lang='en', processors="tokenize,ner", tokenize_no_ssplit=True)

2021-06-17 19:52:48 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-06-17 19:52:48,587 - stanza - INFO - Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-06-17 19:52:48 INFO: Use device: gpu
2021-06-17 19:52:48,589 - stanza - INFO - Use device: gpu
2021-06-17 19:52:48 INFO: Loading: tokenize
2021-06-17 19:52:48,590 - stanza - INFO - Loading: tokenize
2021-06-17 19:52:48 INFO: Loading: ner
2021-06-17 19:52:48,597 - stanza - INFO - Loading: ner
2021-06-17 19:52:49 INFO: Done loading processors!
2021-06-17 19:52:49,096 - stanza - INFO - Done loading processors!


In [23]:
 original_dataframe_names = ['semeval2010_train', 'semeval2010_val']
 makedir(outdir1, out)
 makedir(outdir2, out)
 makedir(outdir3, out)
 makedir(outdir4, out)

In [24]:
for original_df_name in original_dataframe_names:
    type1 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp)
    type2 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 2)
    type3 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 3)
    type4 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 4)
    write_dataframe(type1, out(outdir1 + original_df_name + '_entity_blinding.csv'))
    write_dataframe(type2, out(outdir2 + original_df_name + '_punct_stop_digit.csv'))
    write_dataframe(type3, out(outdir3 + original_df_name + '_punct_digit.csv'))
    write_dataframe(type4, out(outdir4 + original_df_name + '_ner_blinding.csv'))

place_by': 'O', 'insert': None}, '13:15': {'replace_by': 'O', 'insert': None}, '18:24': {'replace_by': 'O', 'insert': None}, '27:30': {'replace_by': 'O', 'insert': None}, '33:38': {'replace_by': 'O', 'insert': None}, '41:44': {'replace_by': 'O', 'insert': None}, '47:47': {'replace_by': 'O', 'insert': None}, '50:57': {'replace_by': 'O', 'insert': None}, '60:63': {'replace_by': 'O', 'insert': None}, '66:70': {'replace_by': 'O', 'insert': None}, '73:74': {'replace_by': 'S-CARDINAL', 'insert': None}, '77:78': {'replace_by': 'O', 'insert': None}, '81:89': {'replace_by': 'O', 'insert': None}, '92:92': {'replace_by': 'O', 'insert': None}, '95:99': {'replace_by': 'O', 'insert': None}, '102:104': {'replace_by': 'O', 'insert': None}, '107:109': {'replace_by': 'O', 'insert': None}, '112:115': {'replace_by': 'O', 'insert': None}, '118:118': {'replace_by': 'O', 'insert': None}, '121:129': {'replace_by': 'O', 'insert': None}, '132:134': {'replace_by': 'O', 'insert': None}, '137:139': {'replace_by': 

In [25]:
 for original_df_name in original_dataframe_names:
     type1 = read_dataframe(out(outdir1 + original_df_name + '_entity_blinding.csv'))
     type2 = read_dataframe(out(outdir2 + original_df_name + '_punct_stop_digit.csv'))
     type3 = read_dataframe(out(outdir3 + original_df_name + '_punct_digit.csv'))
     type4 = read_dataframe(out(outdir4 + original_df_name + '_ner_blinding.csv'))
     write_into_txt(type1, out(outdir1 + original_df_name + '_entity_blinding.txt'))
     write_into_txt(type2, out(outdir2 + original_df_name + '_punct_stop_digit.txt'))
     write_into_txt(type3, out(outdir3 + original_df_name + '_punct_digit.txt'))
     write_into_txt(type4, out(outdir4 + original_df_name + '_ner_blinding.txt'))

100%|██████████| 2717/2717 [00:00<00:00, 5736.40it/s]


In [26]:
write_relations_json(out(''))

In [27]:
def output_file_length(out, filename):
    return len(open(out(filename)).readlines())

In [28]:
print(output_file_length(out, indir + 'semeval2010_train_original.txt'))
print(output_file_length(out, outdir1 + 'semeval2010_train_entity_blinding.txt'))
print(output_file_length(out, outdir2 + 'semeval2010_train_punct_stop_digit.txt'))
print(output_file_length(out, outdir3 + 'semeval2010_train_punct_digit.txt'))
print(output_file_length(out, outdir4 + 'semeval2010_train_ner_blinding.txt'))

8000
8000
8000
8000
7916


In [29]:
print(output_file_length(out, indir + 'semeval2010_val_original.txt'))
print(output_file_length(out, outdir1 + 'semeval2010_val_entity_blinding.txt'))
print(output_file_length(out, outdir2 + 'semeval2010_val_punct_stop_digit.txt'))
print(output_file_length(out, outdir3 + 'semeval2010_val_punct_digit.txt'))
print(output_file_length(out, outdir4 + 'semeval2010_val_ner_blinding.txt'))

2717
2717
2717
2717
2688
